#Attribute Information
1) gender: "Male", "Female" or "Other"
2) age: age of the patient
3) hypertension: 0 if the patient doesn't have hypertension, 1 if the patient has hypertension
4) heart disease: 0 if the patient doesn't have any heart diseases, 1 if the patient has a heart disease 
5) ever-married: "No" or "Yes"
6) worktype: "children", "Govtjov", "Neverworked", "Private" or "Self-employed" 
7) Residencetype: "Rural" or "Urban"
8) avgglucoselevel: average glucose level in blood
9) bmi: body mass index
10) smoking_status: "formerly smoked", "never smoked", "smokes" or "Unknown"*
11) stroke: 1 if the patient had a stroke or 0 if not

In [48]:
import pandas as pd
import sqlite3 as sq

Creating a Database

In [49]:
conn = sq.connect('brainstroke.db')

In [50]:
# Create a table called brainStroke
query = """
CREATE TABLE brainStroke(
    gender VARCHAR(10),
    age DECIMAL(4,1),
    hypertension SMALLINT,
    heart_disease SMALLINT,
    ever_married VARCHAR(5),
    work_type VARCHAR(20),
    residence_type VARCHAR(10),
    avg_glucose_level DECIMAL(5,2),
    bmi DECIMAL(4,1),
    smoking_status VARCHAR(15),
    stroke SMALLINT
)
"""
with conn:
    cur = conn.cursor()
    cur.execute("DROP TABLE IF EXISTS brainStroke")
    cur.execute(query)

In [51]:
# check our sqlite master for confirmation
query = "SELECT name FROM sqlite_master WHERE type='table'"
cur.execute(query)
cur.fetchall()

[('brainStroke',)]

#Loading CSV

In [52]:
brain_df = pd.read_csv('brain_stroke.csv')
brain_df.head()

,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,Male,67.0,0,1,Yes,Private,Urban,228.69,36.6,formerly smoked,1
1,Male,80.0,0,1,Yes,Private,Rural,105.92,32.5,never smoked,1
2,Female,49.0,0,0,Yes,Private,Urban,171.23,34.4,smokes,1
3,Female,79.0,1,0,Yes,Self-employed,Rural,174.12,24.0,never smoked,1
4,Male,81.0,0,0,Yes,Private,Urban,186.21,29.0,formerly smoked,1


In [53]:
brain_df.shape

(4981, 11)

In [54]:
csv_tuple = tuple(brain_df.itertuples(index=False, name=None))
csv_tuple[0]

('Male',
 67.0,
 0,
 1,
 'Yes',
 'Private',
 'Urban',
 228.69,
 36.6,
 'formerly smoked',
 1)

In [55]:
#Insert values into database table
#The no of ? should be equal to the no of columns
insert_query = "INSERT INTO brainStroke VALUES(?,?,?,?,?,?,?,?,?,?,?)"
cur.executemany(insert_query, csv_tuple)

In [56]:
#Limit of the number of rows to show
query = "SELECT * FROM brainStroke LIMIT 5"

In [57]:
pd.read_sql_query(query, conn)

,gender,age,hypertension,heart_disease,ever_married,work_type,residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,Male,67,0,1,Yes,Private,Urban,228.69,36.6,formerly smoked,1
1,Male,80,0,1,Yes,Private,Rural,105.92,32.5,never smoked,1
2,Female,49,0,0,Yes,Private,Urban,171.23,34.4,smokes,1
3,Female,79,1,0,Yes,Self-employed,Rural,174.12,24.0,never smoked,1
4,Male,81,0,0,Yes,Private,Urban,186.21,29.0,formerly smoked,1



###Guide Questions
1. What's the distribution of gender in the dataset?
2. What's the distribution of stroke in the dataset?
3. What's the distribution of brain stroke by gender?
4. For women, how many percent has stroke?
5. For men, how many percent has no stroke?
6. Details of the oldest people with stroke
7. How many people are married and has stroke?
8. How many people are not married and has stroke?
9. What's the minimum, average and maximum age in the data? 

What's the distribution of gender in the dataset?

In [58]:
query = """
SELECT    gender, COUNT(*) count
FROM      brainStroke
GROUP BY  gender
"""
pd.read_sql_query(query, conn)

,gender,count
0,Female,2907
1,Male,2074


2. What's the distribution of stroke in the dataset?

In [59]:
query = """
SELECT    stroke, COUNT(*) count
FROM      brainStroke
GROUP BY  stroke
"""
pd.read_sql_query(query, conn)

,stroke,count
0,0,4733
1,1,248


3. What's the distribution of brain stroke by gender.

In [60]:
query = """
SELECT    gender, stroke, COUNT(*) count
FROM      brainStroke
GROUP BY  gender, stroke
"""
pd.read_sql_query(query, conn)

,gender,stroke,count
0,Female,0,2767
1,Female,1,140
2,Male,0,1966
3,Male,1,108


4. For women, what percentage has stroke?

In [61]:
query = """
SELECT      gender, stroke, CAST(COUNT(*) AS FLOAT)/total.count * 100 AS Percent
FROM        brainStroke, (SELECT COUNT(*) count FROM brainStroke WHERE gender = 'Female') total
WHERE       gender = 'Female'
GROUP BY    stroke

UNION ALL

SELECT      gender, stroke, CAST(COUNT(*) AS FLOAT)/total.count * 100 AS Percent
FROM        brainStroke, (SELECT COUNT(*) count FROM brainStroke WHERE gender = 'Male') total
WHERE       gender = 'Male'
GROUP BY    stroke
"""
pd.read_sql_query(query, conn)

,gender,stroke,Percent
0,Female,0,95.184039
1,Female,1,4.815961
2,Male,0,94.792671
3,Male,1,5.207329


6. Details of the oldest people with stroke

In [62]:
query = """
WITH older_people AS (
    SELECT  *
    FROM    brainStroke
    WHERE   age > 70
)

SELECT  gender, work_type, COUNT(*) count 
FROM    older_people
GROUP BY    gender, work_type
"""
pd.read_sql_query(query, conn)

,gender,work_type,count
0,Female,Govt_job,52
1,Female,Private,193
2,Female,Self-employed,175
3,Male,Govt_job,31
4,Male,Private,143
5,Male,Self-employed,113


7. How many people are married and has stroke?

In [63]:
query = """
SELECT  COUNT(*) Married_and_has_stroke
FROM    brainStroke
WHERE   ever_married = 'Yes' AND stroke = 1
"""
pd.read_sql_query(query, conn)

,Married_and_has_stroke
0,219


8. How many people are not married and has stroke?

In [64]:
query = """
SELECT  COUNT(*) Not_Married_and_has_stroke
FROM    brainStroke
WHERE   ever_married = 'No' AND stroke = 1
"""
pd.read_sql_query(query, conn)

,Not_Married_and_has_stroke
0,29


9. What's the minimum, average and maximum age in the data? 

In [65]:
query = """
SELECT MAX(age) max_age, MIN(age) min_age, AVG(age) avg_age
  FROM brainStroke
"""
pd.read_sql_query(query, conn)

,max_age,min_age,avg_age
0,82,0.08,43.419859


In [66]:
#Further look into the age column
query = """
SELECT *
  FROM brainStroke
 WHERE age < 10
 LIMIT 10
"""
pd.read_sql_query(query, conn)

,gender,age,hypertension,heart_disease,ever_married,work_type,residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,Male,3.00,0,0,No,children,Rural,95.12,18.0,Unknown,0
1,Female,8.00,0,0,No,Private,Urban,110.89,17.6,Unknown,0
2,Female,3.00,0,0,No,children,Urban,73.74,16.0,Unknown,0
3,Male,4.00,0,0,No,children,Rural,79.17,20.0,Unknown,0
4,Male,8.00,0,0,No,children,Rural,78.05,25.7,Unknown,0
5,Female,0.64,0,0,No,children,Urban,83.82,24.9,Unknown,0
6,Female,4.00,0,0,No,children,Rural,93.25,16.6,Unknown,0
7,Female,0.88,0,0,No,children,Rural,88.11,15.5,Unknown,0
8,Male,4.00,0,0,No,children,Rural,106.22,16.7,Unknown,0
9,Male,5.00,0,0,No,children,Rural,85.84,16.4,Unknown,0


10. Children with either hypertension or stroke.

In [67]:
query = """
SELECT  *
FROM    brainStroke
WHERE   work_type = 'children' AND (hypertension = 1 OR stroke =1)
"""
pd.read_sql_query(query, conn)

,gender,age,hypertension,heart_disease,ever_married,work_type,residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,Female,14.00,0,0,No,children,Rural,57.93,30.9,Unknown,1
1,Female,1.32,0,0,No,children,Urban,70.37,18.7,Unknown,1


#TASK

1. What's the distibution of hypertension in the dataset?

In [68]:
query = """
SELECT    hypertension, COUNT(*) count
FROM      brainStroke
GROUP BY  hypertension
"""
pd.read_sql_query(query, conn)

,hypertension,count
0,0,4502
1,1,479


2. How many males and females have hypertension?

In [69]:
query = """
SELECT    gender, hypertension, COUNT(*) count
FROM      brainStroke
GROUP BY  gender, hypertension
"""
pd.read_sql_query(query, conn)

,gender,hypertension,count
0,Female,0,2643
1,Female,1,264
2,Male,0,1859
3,Male,1,215


3. For each smoking status show how many people have hypertension and how many do not.

In [70]:
query = """
SELECT    smoking_status, hypertension, COUNT(*) count
FROM      brainStroke
GROUP BY  smoking_status, hypertension
"""
pd.read_sql_query(query, conn)

,smoking_status,hypertension,count
0,Unknown,0,1450
1,Unknown,1,50
2,formerly smoked,0,752
3,formerly smoked,1,115
4,never smoked,0,1615
5,never smoked,1,223
6,smokes,0,685
7,smokes,1,91


4. How many males and females have both hypertension and heart disease?

In [71]:
#Males with both hypertension and heart disease
query = """
SELECT  COUNT(*)count
FROM    brainStroke
WHERE   gender = 'Male' AND (hypertension = 1 AND  heart_disease = 1)
"""
pd.read_sql_query(query, conn)




,count
0,35


In [72]:
#Females with both hypertension and heart disease
query = """
SELECT  COUNT(*)count
FROM    brainStroke
WHERE   gender = 'Female' AND (hypertension = 1 AND  heart_disease = 1)
"""
pd.read_sql_query(query, conn)

,count
0,29


In [73]:
#This gives a df  
query = """
SELECT  *
FROM    brainStroke
WHERE   gender = 'Female' AND (hypertension = 1 AND  heart_disease = 1)
"""
pd.read_sql_query(query, conn)

,gender,age,hypertension,heart_disease,ever_married,work_type,residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,Female,82,1,1,No,Private,Rural,84.03,26.5,formerly smoked,1
1,Female,69,1,1,No,Self-employed,Urban,72.17,36.8,never smoked,1
2,Female,82,1,1,Yes,Govt_job,Urban,215.94,27.9,formerly smoked,1
3,Female,55,1,1,Yes,Private,Urban,210.40,40.0,smokes,1
4,Female,68,1,1,Yes,Private,Urban,247.51,40.5,formerly smoked,1
5,Female,79,1,1,Yes,Self-employed,Urban,200.28,30.0,formerly smoked,0
6,Female,80,1,1,Yes,Private,Rural,115.52,34.4,Unknown,0
7,Female,78,1,1,Yes,Private,Urban,227.16,41.7,never smoked,0
8,Female,76,1,1,Yes,Private,Rural,102.08,31.0,smokes,0
9,Female,53,1,1,Yes,Private,Urban,196.25,24.9,smokes,0


5a. Which residence has the lowest number of people with no heart disease?

In [74]:
query = """
SELECT  residence_type, COUNT(*)count
FROM    brainStroke
WHERE   (heart_disease = 0)
GROUP BY residence_type
"""
pd.read_sql_query(query, conn)

,residence_type,count
0,Rural,2315
1,Urban,2391


5b. Which residence has the lowest number of people with  heart disease?

In [75]:
query = """
SELECT  residence_type, COUNT(*)count
FROM    brainStroke
WHERE   (heart_disease = 1)
GROUP BY residence_type
"""
pd.read_sql_query(query, conn)

,residence_type,count
0,Rural,134
1,Urban,141


6a.  What percentage of people with private work type have no hypertension?

In [79]:
query = """
SELECT      work_type, hypertension, CAST(COUNT(*) AS FLOAT)/total.count * 100 AS Percent
FROM        brainStroke, (SELECT COUNT(*) count FROM brainStroke WHERE work_type = 'Private') total
WHERE       work_type = 'Private'
GROUP BY    hypertension
"""
pd.read_sql_query(query, conn)

,work_type,hypertension,Percent
0,Private,0,90.48951
1,Private,1,9.51049


6b.  What percentage of people with Government job type have no hypertension?

In [83]:
query = """
SELECT      work_type, hypertension, CAST(COUNT(*) AS FLOAT)/total.count * 100 AS Percent
FROM        brainStroke, (SELECT COUNT(*) count FROM brainStroke WHERE work_type = 'Govt_job') total
WHERE       work_type = 'Govt_job'
GROUP BY    hypertension
"""
pd.read_sql_query(query, conn)

,work_type,hypertension,Percent
0,Govt_job,0,89.130435
1,Govt_job,1,10.869565


6c.  What percentage of people with Self-employed job type have no hypertension?

In [82]:
query = """
SELECT      work_type, hypertension, CAST(COUNT(*) AS FLOAT)/total.count * 100 AS Percent
FROM        brainStroke, (SELECT COUNT(*) count FROM brainStroke WHERE work_type = 'Self-employed') total
WHERE       work_type = 'Self-employed'
GROUP BY    hypertension
"""
pd.read_sql_query(query, conn)

,work_type,hypertension,Percent
0,Self-employed,0,82.960199
1,Self-employed,1,17.039801


### Summary
1. There are more Females than Males, 2,907 and 2,074 respectively.
2. 4,733 persons have no Stroke while the remaining 248 have Stroke.
4. 5% of Females have Stroke.
5. There are more elderly people in the private sector than other work types.
6. 219 Married people have Stroke.
7. Two children have Stroke.
8. 4502 people have no hypertension while 479 have hypertension.
9. More Females (264) have hypertension as compared to males(215).
10. 91 smokers have hypertension, 115 people who formely smoked have hypertension while 223 people who have never smoked have hypertension.
11. 35 Males and 29 females have both hypertension and heart disease.
12. While rural residences have a lower number of people with heart disease, Urban residnces have a higher number of people without heart disease.
13. Comparing percentages of people with hypertension in Private, Self-employed and Government job types, private job type is the best. 